Create a histogram showing the distribution of cloud band area for 2 periods: DJFM and full years

Run: python cloudbandPy/analysis/run_analysis_plot_histogram_cb_area.py cloudbandPy/config/config_analysis.yml

In [7]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os, sys


from cloudbandpy.figure_tools import set_fontsize
from cloudbandpy.io_utilities import load_ymlfile, load_data_from_saved_var_files
from cloudbandpy.misc import parse_arguments
from cloudbandpy.time_utilities import add_startend_datetime2config

Functions for bins for histograms

In [8]:
def create_bins(min_value=0.01, max_value=100, stepbins=5):
    bins = np.linspace(min_value, max_value, stepbins)
    return bins


def pdfbinned(bins, variable):
    qt = [0] * bins
    for idx, q in enumerate(bins):
        if idx < np.size(bins) - 1:
            qabove = bins[idx + 1]
            pts = [i for i in range(len(variable)) if q <= variable[i] < qabove]
            Ps = [variable[p] for p in pts]
        elif idx == np.size(bins) - 1:
            continue
        qt[idx] = np.size(Ps) / float(len(variable))
    return qt

Load cloud bands

In [12]:
base_dir = "../"

config_file = os.path.join(base_dir, "config", "config_cbworkflow_northernhemisphere.yml")
config = load_ymlfile(config_file, isconfigfile=True)
# Make sure the dates are covering the whole period
config_copy = config.copy()
config_copy["startdate"] = "19590101.00"
config_copy["enddate"] = "20211231.00"
add_startend_datetime2config(config_copy)

list_of_cloud_bandsSP = load_data_from_saved_var_files(config_copy, varname="list_of_cloud_bands")
# flatten list of cloud band areas
# cb_areaSP = [cb.area for sublist in list_of_cloud_bandsSP for cb in sublist]


2023-12-08 16:08:57,171 - io_utilities.load_ymlfile - INFO: Configuration file loaded
2023-12-08 16:08:57,172 - io_utilities.load_data_from_saved_var_files - WARNING: Check your config file and if you have saved any data


Function for figure

In [4]:
def plot_distribution_cb_area(cbarea1, cbarea2):
    # Distribution
    bins = create_bins(min_value=min(cbarea1), max_value=max(cbarea1), stepbins=100)
    qt1 = pdfbinned(bins, cbarea1)
    qt2 = pdfbinned(bins, cbarea2)
    # Figure
    set_fontsize(size=14)
    mpl.rcParams["axes.spines.left"] = True
    mpl.rcParams["axes.spines.right"] = False
    mpl.rcParams["axes.spines.top"] = False
    mpl.rcParams["axes.spines.bottom"] = True
    #
    yearstart = config["datetime_startdate"].year
    yearend = config["datetime_enddate"].year
    fig, ax = plt.subplots()
    ax.plot(bins, qt1, color="#3abc76", label=f"{yearstart}-{yearend}")
    ax.plot(bins, qt2, color="#3f4d89", label=f"{yearstart}-{yearend} DJFM")
    #
    ax.set_yticks([el * 1e-2 for el in np.arange(0, 4.5, 0.5)])
    ax.set_yticklabels([f"{el:2.1f}" for el in [el for el in np.arange(0, 4.5, 0.5)]])
    ax.set_xticks([el * 1e7 for el in np.arange(0.25, 2.5, 0.25)])
    ax.set_xticklabels([f"{el:3.1f}" for el in [el * 10 for el in np.arange(0.25, 2.5, 0.25)]])
    ax.set(xlabel=r"Cloud band area (10$^6\,$m$^2$)", ylabel=r"Standardized distribution (10$^{-2}$)")
    plt.legend(framealpha=0.0)
    return fig